# Swarm_Dst 科研手册

## 0. 本机的conda环境选择
Swarm_Dst

pytorch



## 1. 数据读取操作

    对下载到的SWarm数据与Dst数据进行预处理，Swarm数据下载方法见readme.ipynb，此次下载的Swarm数据中存在有卫星提供的Dst值，
    同时，也有从地磁台站下载解算得到的Dst值，两者存在差异，此次实验将会对两种Dst数值均进行实验解算。

Swarm_Dst_finished脚本功能：
读取Swarm卫星数据，并进行数据筛选，筛选出特定经纬度范围内，以及特定时间段内的数据
1. 经纬度手动调整，参考依据：文献Fast Dst computation by applying deep learning to Swarm satellite magnetic data
2. 特定时间段，最安静的10天+最受干扰的5天，参考依据：文献Fast Dst computation by applying deep learning to Swarm satellite magnetic data + 
   从世界地磁数据中心下载的QDdays数据，格式见网站。
3. 输出数据文件保存至Swarm_select.npy，目前仅有SWarm提供的Dst数值，并未加入台站测量的Dst值  !!!结合有关产品手册来看两者的dst值应该是一样的
4. 'Spacecraft' 'Timestamp', 'Latitude', 'Longitude', 'Radius', 'Dst', 'B_NEC_CHAOS-internal', 'QDLon', 'QDLat', 'B_NEC'
5. !!! 环境选用Swarm_Dst

In [ ]:
globals().clear()
import cdflib
import os
from matplotlib.dates import YearLocator
import numpy as np
import myfunction
import logging


#----------------------确定筛选参数
belt_width_lat = 0.2
lat_cen        = [-40,-30,-20,20,30,40]  # 单位：度

belt_width_lon = 360.0
lon_cen        = [0]

Swarm_dir      = "Swarm_Data/"
Dst_index      = "Dst_index/index.dat"
QDday          = "QDday/QDday.txt"
select_dir     = "Swarm_select/"

#----------------------读取QDdays文件并保存到相应数据矩阵中

QDday_data = myfunction.QDread(QDday)
QDday_data = np.delete(QDday_data,range(0,11),axis=0)  # 将QDday数据的时间轴与Swarm数据对齐

#--------------------- Create a output logging
logger = logging.getLogger('mylogger')
logger.setLevel(logging.DEBUG)
logger_name = 'my_log.txt'
fh = logging.FileHandler(logger_name)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

#-----------------------创建保存输出文件的文件夹
if not os.path.exists(select_dir):
    os.makedirs(select_dir)


#----------------------获取所有数据的文件名
files = sorted(os.listdir(Swarm_dir))   # sorted函数用于排序


#------------------------文件戳 记录第几个文件
file_epoch = 1

for file_name in files:  
    file_path = Swarm_dir + file_name
    cdf_file = cdflib.CDF(file_path)
    var = cdf_file.cdf_info()
    #----------------------针对每一个文件中的数据进行筛选 
    data_mat_finish = np.ones((1,14))   # 初始化数据保存矩阵，对数据文件中的每一行进行筛选判断，若满足条件则加入其中 第一行是无意义的1，后续需要删除

    #-----------------------------

    raw_data = cdf_file.varget(var.zVariables[0])   #读入数据文件中的所有初始数据
    for i in var.zVariables[1:]:
        data0 = cdf_file.varget(i)
        raw_data = np.column_stack((raw_data,data0))

    for i in range(0,raw_data.shape[0]):           # 进行数据筛选
        raw_data_test   = raw_data[i,:].reshape(1,-1)
        lat             = float(raw_data_test[0,2])
        lon             = float(raw_data_test[0,3])
        time_           = cdflib.cdfepoch.to_datetime(float(raw_data_test[0,1]))
        year            = int(str(time_[0])[0:4])
        mon             = int(str(time_[0])[5:7])
        day             = int(str(time_[0])[8:10])


        QDday_data_test = QDday_data[file_epoch-1,:].reshape(1,-1)
        QD_year         = QDday_data_test[0,0]
        QD_mon          = QDday_data_test[0,1]
        
        if (year != QD_year) or (mon != QD_mon) :   # 判断时间轴是否对齐，否则跳出循环
            print(f"{year}-{mon}Time is wrong!!!")
            break
        
        if not (day in QDday_data_test[0,2:17]):
            continue
        elif not myfunction.location_pd(lat,belt_width_lat,lat_cen):
            continue
        elif not myfunction.location_pd(lon,belt_width_lon,lon_cen):
            continue
        
        data_mat_finish = np.row_stack((data_mat_finish,raw_data_test))

    log_message    =  file_name+"has been calculated."
    logger.debug(log_message)
    #-----------------------------------------------
    file_epoch += 1
        
    #---------------------------------------------------save data
    data_mat_finish = np.delete(data_mat_finish,0,axis=0)   # 第一行需要删除
    save_file       = select_dir + 'Swarm_data_' + str(year).zfill(4) + str(mon).zfill(2) + '_finishedtest.npy'
    np.save(save_file,data_mat_finish)     


## 2. 数据预处理
    从卫星观测值中扣除地磁场内部贡献（地核+地壳）


Swarm_Dst_process脚本功能：
读取筛选后的Swarm卫星数据（/Swarm_select），并进行数据预处理，扣除地磁场内部影响，并输出后续深度学习所需要的数据格式,保存至"train_valid_database/"文件夹
1. 选择以下7个变量作为网络的输入“特征”：地磁纬度、地磁经度、地磁当地时间(MLat, MLon and MLT)，卫星高度以及三个地磁分量残差(res. X, Y和Z)。参考依据：文献Fast Dst computation 
   by applying deep learning to Swarm satellite magnetic data
2. 筛选出训练集、验证集，本次实验的思路是：使用交叉验证法，分成10组，同时由于Dst的特性，有实时Dst值，临时Dst值和最终Dst值（三者的计算方法和详细定义见Version definition Dst）
   尽量保证三种Dst值的训练集和验证集分布相同   ！！！由于Dst会定期更新，截止本次实验，最终Dst值（~2016-12）、临时Dst值（2017-01~2023-06）、实时Dst值（2023-07~）
3. 输出数据文件夹"train_valid_database/"
4. 'Spacecraft' 'Timestamp', 'Latitude', 'Longitude', 'Radius', 'Dst', 'B_NEC_CHAOS-internal', 'QDLon', 'QDLat', 'B_NEC'
5. !!! 环境选用Swarm_Dst

In [1]:
globals().clear()
import os
import cdflib
import numpy as np
import myfunction
import logging


#----------------------确定输入参数
select_dir             = "Swarm_select/"
groups_num             = 10     # 交叉验证法的总组数

final_Dst_begin        = [2013,12,1,0,0,0,0]   # 数字含义：年。月，日，时，分，秒，毫秒
final_Dst_end          = [2016,12,31,23,59,59,999]
provisional_Dst_begin  = [2017,1,1,0,0,0,0]   # 数字含义：年。月，日，时，分，秒，毫秒
provisional_Dst_end    = [2023,6,30,23,59,59,999]
real_time_Dst_begin    = [2023,7,1,0,0,0,0]   # 数字含义：年。月，日，时，分，秒，毫秒
real_time_Dst_end      = [2023,12,31,23,59,59,999]

train_valid_data_dir   = "train_valid_database/"

#--------------------- Create a output logging
logger = logging.getLogger('mylogger')
logger.setLevel(logging.DEBUG)
logger_name = 'my_log.txt'
fh = logging.FileHandler(logger_name)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

#-----------------------创建保存输出文件的文件夹
if not os.path.exists(train_valid_data_dir):
    os.makedirs(train_valid_data_dir)

#----------------------获取所有数据的文件名
files = sorted(os.listdir(select_dir))   # sorted函数用于排序

#----------------------读取筛选得到Swarm_select数据，进行汇总与分类.

final_mat          = np.zeros((1,14))    # 初始化数据保存矩阵，对数据文件中的每一行进行筛选判断，若满足条件则加入其中 第一行是无意义的0，后续需要删除
provisional_mat    = np.zeros((1,14))
real_time_mat      = np.zeros((1,14))


for file_name in files:
    data_matrix = np.load(select_dir+file_name)
    time_epoch  = data_matrix[0,1]
    if (float(time_epoch) >= cdflib.cdfepoch.compute_epoch(final_Dst_begin)) and (float(time_epoch) <= cdflib.cdfepoch.compute_epoch(final_Dst_end)) :
        final_mat           =  np.vstack((final_mat,data_matrix))
        continue
    elif (float(time_epoch) >= cdflib.cdfepoch.compute_epoch(provisional_Dst_begin)) and (float(time_epoch) <= cdflib.cdfepoch.compute_epoch(provisional_Dst_end)) :
        provisional_mat     =  np.vstack((provisional_mat,data_matrix))
        continue
    elif (float(time_epoch) >= cdflib.cdfepoch.compute_epoch(real_time_Dst_begin)) and (float(time_epoch) <= cdflib.cdfepoch.compute_epoch(real_time_Dst_end)) :
        real_time_mat       =  np.vstack((real_time_mat,data_matrix))
        continue
    
    log_message    =  file_name+"has been calculated."
    logger.debug(log_message)


#---------------------------------------------------save data
final_mat        = np.delete(final_mat,0,axis=0)   # 第一行需要删除
provisional_mat  = np.delete(provisional_mat,0,axis=0)
real_time_mat    = np.delete(real_time_mat,0,axis=0)

save_file        = train_valid_data_dir + 'final_mat.npy'
np.save(save_file,final_mat)    
save_file        = train_valid_data_dir + 'provisional_mat.npy'
np.save(save_file,provisional_mat)
save_file        = train_valid_data_dir + 'real_time_mat.npy'
np.save(save_file,real_time_mat) 



## 3. 深度学习框架的搭建

    此步骤环境选用pytorch

以下为测试脚本。不可放进主程序运行！！！！！！

In [3]:
import multiprocessing

print(multiprocessing.cpu_count())

24
